# Test Notebook

This is mainly a test notebook that runs db operations on the small test db (DEV connection) that is first created, then filled with 3 records and 2 'addition' credit trx, before a record is deactivated and then also fully deleted. Assert statements help to check everything works as expected.

But you can also use it to test front end manipulations if you run the front end against the DEV db. Use parts 2 / 3 for that after having run the notebook once from start to end.

In [1]:
import configparser
import datetime as dt
import sys
import collections
from pathlib import Path
from typing import Union, Optional, Iterable, List, Tuple

import codebook.EDA as EDA
import codebook.clean as clean
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlalchemy
from sqlalchemy import func, distinct

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
plt.style.use('raph-base')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', 30)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

np.random.seed(666)

In [3]:
sys.path.append(str(Path.cwd().parent))

from src.db_declaration import (
    Base, Artist, CreditTrx, Genre, Label, Record, RecordFormat, 
    ArtistRecordLink, ArtistGenreLink, GenreLabelLink, LabelRecordLink
)
from src import db_functions
from src import db_connect

CONFIG_PATH = Path.cwd().parent / "config.yaml"

In [4]:
print(sys.executable)
print(sys.version)
print(f'sqlalchemy {sqlalchemy.__version__}')

C:\Users\r2d4\miniconda3\envs\py3\python.exe
3.8.3 (default, May 19 2020, 06:50:17) [MSC v.1916 64 bit (AMD64)]
sqlalchemy 1.3.17


## (Re-)Create A Test DB

### Connect And Reset

Because we connect from `dev`subfolder, I work with the sqlalchemy `create_engine` function directly.

In [5]:
# session.close()

db_params = {"REL_PATH": "DeafDiscoBase.db"}
engine = db_connect.create_engine(db_params)
session = db_connect.create_session(engine)

# Check
engine
session

Engine(sqlite:///C:\Users\r2d4\OneDrive\code\projects\20-02_disco\dev\DeafDiscoBase.db)

In [6]:
db_functions._drop_and_reset_DB(engine, Base)

### Load and Insert 3 Test Records, assert

In [7]:
record_data = pd.read_parquet(
    r"C:\Users\r2d4\OneDrive\code\projects\20-02_disco\db_aka_discobase\back-up\record_data_2021-02-17-09-02-53.parquet"
)
test_records = record_data[record_data.index.isin([1, 225, 301])].copy()
test_records.head()

,artist,artist_country,title,genre,label,year,record_format,vinyl_color,lim_edition,number,remarks,purchase_date,price,rating,is_digitized,is_active
record_id,,,,,,,,,,,,,,,,
1,[Dismember],[NA],Pieces,Death Metal,[NA],1992,"12""",None,None,None,None,1992-01-01,35.00,nan,False,True
225,[Petrification],[NA],Hollow of The Void,Death Metal,[Dawnbreed / Sentient Ruin],2018,LP,green transparent with black haze,200,None,None,2019-07-01,25.00,10.00,True,True
301,"[Coffins, Depression]","[NA, Germany]",Split,Death Metal,[Hells Headbangers],2021,LP,white with black splatter,100,None,None,2021-01-20,20.00,nan,True,True


In [8]:
db_functions._insert_record_data_with_sqlalchemy_orm(session, test_records)

C:\Users\r2d4\miniconda3\envs\py3\lib\site-packages\sqlalchemy\sql\sqltypes.py:660: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  util.warn(


In [9]:
assert session.query(Record).count() == 3
assert session.query(ArtistRecordLink).count() == 4

### Insertion of 2 Credit Addition Trx, assert

In [10]:
# to_delete = session.query(CreditTrx).filter(CreditTrx.credit_trx_id == 8).one()
# session.delete(to_delete)
# session.commit()

In [11]:
# Initial trx, 11 days ago

addition_trx = CreditTrx(
    credit_trx_date=dt.datetime.today().date() - dt.timedelta(11),
    credit_trx_type="Addition",
    credit_value=1,
    credit_saldo=1,
    record_id=np.nan
)
session.add(addition_trx)

# And a regular interval addition
db_functions.add_regular_credits(session)

session.commit()

Creating 'Addition' Trx for: 2021-02-21


In [12]:
pd.read_sql("credit_trx", engine)

,credit_trx_id,credit_trx_date,credit_trx_type,credit_value,credit_saldo,record_id,created_at,updated_at
0,1,1992-01-01,Initial Load,0.00,0.00,1.00,2021-02-22 13:42:51,NaT
1,2,2019-07-01,Initial Load,0.00,0.00,2.00,2021-02-22 13:42:51,NaT
2,3,2021-01-20,Initial Load,0.00,0.00,3.00,2021-02-22 13:42:51,NaT
3,4,2021-02-11,Addition,1.00,1.00,nan,2021-02-22 13:43:03,NaT
4,5,2021-02-21,Addition,1.00,2.00,nan,2021-02-22 13:43:03,NaT


In [13]:
assert session.query(CreditTrx).count()
assert session.query(func.sum(CreditTrx.credit_value)).all()[0][0] == 2
assert session.query(func.avg(CreditTrx.credit_value)).all()[0][0] == 0.4
assert session.query(func.max(CreditTrx.credit_saldo)).all()[0][0] == 2

## Test Front-end Manipulations (Optional)

In [14]:
pd.read_sql("artists", engine)

,artist_id,artist_name,artist_country,created_at,updated_at
0,1,Dismember,NA,2021-02-22 13:42:51,NaT
1,2,Petrification,NA,2021-02-22 13:42:51,NaT
2,3,Coffins,NA,2021-02-22 13:42:51,NaT
3,4,Depression,Germany,2021-02-22 13:42:51,NaT


In [15]:
pd.read_sql("labels", engine)

,label_id,label_name,created_at,updated_at
0,1,NA,2021-02-22 13:42:51,NaT
1,2,Dawnbreed / Sentient Ruin,2021-02-22 13:42:51,NaT
2,3,Hells Headbangers,2021-02-22 13:42:51,NaT


In [16]:
pd.read_sql("artist_label_link", engine)

,artist_id,label_id
0,1,1
1,2,2
2,3,3
3,4,3


In [17]:
pd.read_sql("artist_label_link", engine)

,artist_id,label_id
0,1,1
1,2,2
2,3,3
3,4,3


In [18]:
session.close()

### Removal of Existing Record, assert

Necessary cols: trx_type, credit_value, title, artist, date

In [18]:
test_removal = {
    "trx_type": "Remove",
    "credit_value": 1,
    "artist": "Coffins",
    "title": "Split",
#     "year": 1993,
    "removal_date": dt.datetime.today().date()
}

In [30]:
db_functions.set_record_to_inactive(session, test_removal)
assert session.query(func.count(distinct(Record.is_active))).all()[0][0] == 2
assert len(session.query(CreditTrx).all()) == 6

Status of record 'Split' by Coffins is already 0, please check.


In [22]:
# session.query(CreditTrx).all()[-3:]

### Reactivation of inactive Record [OPEN]

In [23]:
# ATTENTION It has to be possible ro re-add inactive records! (and to pay for it in credits!)

## Query DB

### Check Tables

In [31]:
pd.read_sql("records", engine)

,record_id,title,year,genre_id,format_id,vinyl_color,lim_edition,number,remarks,purchase_date,price,rating,is_digitized,is_active,created_at,updated_at
0,1,Pieces,1992,1,1,None,None,None,None,1992-01-01,35,nan,0,1,2021-02-22 13:42:51,NaT
1,2,Hollow of The Void,2018,1,2,green transparent with black haze,200,None,None,2019-07-01,25,10.00,1,1,2021-02-22 13:42:51,NaT
2,3,Split,2021,1,2,white with black splatter,100,None,None,2021-01-20,20,nan,1,0,2021-02-22 13:42:51,2021-02-22 13:45:05


In [32]:
pd.read_sql("artists", engine)

,artist_id,artist_name,artist_country,created_at,updated_at
0,1,Dismember,NA,2021-02-22 13:42:51,NaT
1,2,Petrification,NA,2021-02-22 13:42:51,NaT
2,3,Coffins,NA,2021-02-22 13:42:51,NaT
3,4,Depression,Germany,2021-02-22 13:42:51,NaT


In [33]:
pd.read_sql("artist_record_link", engine)

,artist_id,record_id
0,1,1
1,2,2
2,3,3
3,4,3


In [34]:
pd.read_sql("genres", engine)

,genre_id,genre_name,created_at,updated_at
0,1,Death Metal,2021-02-22 13:42:51,NaT


In [35]:
pd.read_sql("artist_genre_link", engine)

,artist_id,genre_id
0,1,1
1,2,1
2,3,1
3,4,1


In [36]:
pd.read_sql("formats", engine)

,format_id,format_name,created_at,updated_at
0,1,"12""",2021-02-22 13:42:51,NaT
1,2,LP,2021-02-22 13:42:51,NaT


In [37]:
pd.read_sql("labels", engine)

,label_id,label_name,created_at,updated_at
0,1,NA,2021-02-22 13:42:51,NaT
1,2,Dawnbreed / Sentient Ruin,2021-02-22 13:42:51,NaT
2,3,Hells Headbangers,2021-02-22 13:42:51,NaT


In [38]:
pd.read_sql("label_record_link", engine)

,label_id,record_id
0,1,1
1,2,2
2,3,3


In [39]:
pd.read_sql("artist_label_link", engine)

,artist_id,label_id
0,1,1
1,2,2
2,3,3
3,4,3


## (Effective) DELETION of Record, assert

In [45]:
to_delete = session.query(Record).filter(Record.record_id == 3).one_or_none()
session.delete(to_delete)
session.commit()

In [48]:
pd.read_sql("labels", engine)
pd.read_sql("artist_label_link", engine)
pd.read_sql("credit_trx", engine)

assert session.query(Record).count() == 1
assert session.query(ArtistRecordLink).count() == 2
assert session.query(CreditTrx).count() == 4

,label_id,label_name,created_at,updated_at
0,1,NA,2021-02-22 13:42:51,NaT
1,2,Dawnbreed / Sentient Ruin,2021-02-22 13:42:51,NaT


,artist_id,label_id
0,1,1
1,2,2


,credit_trx_id,credit_trx_date,credit_trx_type,credit_value,credit_saldo,record_id,created_at,updated_at
0,1,1992-01-01,Initial Load,0.00,0.00,1.00,2021-02-22 13:42:51,NaT
1,2,2019-07-01,Initial Load,0.00,0.00,2.00,2021-02-22 13:42:51,NaT
2,4,2021-02-11,Addition,1.00,1.00,nan,2021-02-22 13:43:03,NaT
3,5,2021-02-21,Addition,1.00,2.00,nan,2021-02-22 13:43:03,NaT
